In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [1]:
image_path = "binary_image.png"
binary_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

NameError: name 'cv2' is not defined